# Compute source time course (STC) from Raw and Epochs data

In [1]:
import sys
import os
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.sl_utils as sl_utils 
from autoreject import AutoReject

# Load in subject IDs and paths from CFGLog
config = Config.from_json(configs.CFGLog)
RANDOM_STATE = config.parameters.random_seed

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC

# CLBP already processed
clbp_processed_path = config.data.chronic_low_back_pain.processed_path

# Define preprocessing parameters
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain'])}")
print(sub_ids)

event_id = configs.CFGLog['parameters']['event_id']
# tmin, tmax = config.parameters.tmin, config.parameters.tmax

average_dipoles = True

imported src module
imported preprocessing module
imported utils module
imported configs module
0 files missing from root.txt in /home/wanglab/mne_data/MNE-fsaverage-data
0 files missing from bem.txt in /home/wanglab/mne_data/MNE-fsaverage-data/fsaverage
Total subjects: 43
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.']}


In [2]:
sub_ids = {
    "chronic_low_back_pain": ['039']
}

In [3]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
                
        subject.load_eyes_open()
        subject.eyes_open.set_eeg_reference(ref_channels='average',
                                            projection=True)
        
        # Load epochs 
        subject.load_epochs()
        subject.epochs.set_eeg_reference(ref_channels='average',
                                            projection=True)
        
        # Source localize
        subject.get_stc_epochs(
            average_dipoles=True,
            method='MNE',
            return_eyes_open=False,
        )
                        
        # Save
        subject.save(subject.stc_epochs, "stc_epochs")
        subject.save(subject.stc_epochs,
                        "stc_epochs", 
                        as_mat=True,
                        overwrite=True,
                        )
        
        # Concatenate epochs into VHDR for visualization
        subject.concatenate_epochs(
            data=subject.stc_epochs, 
            save=True, 
            overwrite=True)

/tmp/ipykernel_37769/869727834.py:6: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  subject.eyes_open.set_eeg_reference(ref_channels='average',
/tmp/ipykernel_37769/869727834.py:11: RuntimeWarning: An average reference projection was already added. The data has been left untouched.
  subject.epochs.set_eeg_reference(ref_channels='average',


Loaded epochs for subject 039
Saved stc_epochs to /home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/data/preprocessed/039_stc_epochs.pkl.
*stc_epochs shape =  (48, 12, 1801)
Saving stc.mat for 039 in region: rostralanteriorcingulate-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: caudalanteriorcingulate-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: postcentral-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: insula-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: superiorfrontal-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: medialorbitofrontal-lh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: rostralanteriorcingulate-rh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: caudalanteriorcingulate-rh
*stc_epochs_i shape =  (48, 1801)
Saving stc.mat for 039 in region: postcentral-rh
*stc_epochs_i shape =  (

/home/wanglab/Documents/George_Kenefati_6.20.24-Present/High-Pain-Cross-Study-EEG/src/preprocessing/classes.py:223: RuntimeWarning: Encountered data in 'double' format. Converting to float32.
  data_object.export(save_file_path, overwrite=overwrite)
